# MESA GYRE SdB Asteroseismic Fitting Tool
The fitting tool uses sdb grids created by Ostrowski et al. 2022 </br>
https://ui.adsabs.harvard.edu/abs/2021MNRAS.503.4646O/abstract </br>

- The grids information is extracted from the main grid database to reduce the file size. <br>
- The module for extracting the grid is 'GridExtract.py' located in the same directory<br> 
- It is saved as a sdb_grid.zip file along with the notebook. <br>
- Please extract the 'sdb_grid.zip' to the same folder where the notbook is located.<br>
- It will create a folder 'sdb_grid' and has all necessary files with in to run the notebook'<br>
- The 'sdb_grid' directory contains all the folders of the etracted grids and 'sdb_grid_extract.csv' file.

<br>

The grid parameters are as follows.

Metalicity (z) => 0.005 to 0.035 </br>
Metalicity (Fe/H) = > -0.46 to +0.39 </br>
Initial Mass (Mi) => 1.0 to 1.8 </br>
Envelope Mass (Menv) => 0.00001 to 0.01 </br>
Central Helium (coreHe) => 0.1 to 0.9 </br>


## Install all dependencies

In [ ]:
! pip install bokeh
! pip install numpy
! pip install pandas
! pip install tqdm

## sdBMESAGYRE python class
This class has mainly four modules. <br>
* ```ExtractGrid``` to extract grid from the main sdB model grid from Ostrowski et al. 2022. It is not necessary to use this module since the grid is already extracted. (Not added in the class Obj for now)
* ```FitModels``` to fit the grid to observations
* ```CheckModels``` to check the fitted models with custom periods
* ```PlotModels``` to plot and visualize the filtered models with the observations.<br>
Load these module by running the cell below

In [1]:
import glob
import pandas as pd
import sys
import numpy as np
import os
from tqdm import tqdm
import timeit
import warnings

from bokeh.io import output_file, show
from bokeh.models import HoverTool, ColumnDataSource, HTMLTemplateFormatter, TableColumn, DataTable, ColorBar, Label, Div, Legend
from bokeh.plotting import figure
from bokeh.models.annotations import Title
from bokeh.layouts import layout, column
from bokeh.transform import linear_cmap
from bokeh.palettes import OrRd4, Plasma256, Viridis10, Magma10, Spectral6

warnings.filterwarnings("ignore")  

class sdBMESAGYRE():
    def __init__(self,Target,ChunkSize=1000, CustomPeriodList = [],HowBad = 150, NModels=1000):
        self.Target = Target
        self.ChunkSize = ChunkSize
        self.CustomPeriodList = CustomPeriodList
        self.HowBad = HowBad
        self.NModels = NModels
        
    def Merit(self,table_gyre,lval):
        ''' Merit fuctions
            Calculates the nearest model periods to the observation.
            Input
            table_gyre (data frame): data from the gyre modules
            p_obs (list) : list of observed periods.
            Return (nparray) the model period differences
        '''
        table_gyre_l1 = table_gyre[table_gyre.l==int(lval)]
        ds = np.zeros(1)
        dprd = np.zeros(1)
        dnval = np.zeros(1)
        for i in self.Target['PeriodList']:
            dP = (table_gyre_l1['P']-i)**2 #period difference square
            table_gyre_l1['dP'] = dP
            table_gyre_period = table_gyre_l1[table_gyre_l1['dP']==min(dP)].P
            table_gyre_nval = table_gyre_l1[table_gyre_l1['dP']==min(dP)].n
            
            dP = min(dP)
            ds = np.vstack((ds,dP))
            dprd = np.vstack((dprd,table_gyre_period))
            dnval = np.vstack((dnval, table_gyre_nval))
        return ds[1:],dprd[1:], dnval[1:]

    def AppendTaskCSV(self):
        '''To append all the fitted csv files to a single csv file'''
        all_files = glob.glob(self.Target['Name']+'_output/'+self.Target['Name']+'_fitgrid_*.csv')
        all_files = sorted(all_files)
        li = []
        for filename in tqdm(all_files):
            df = pd.read_csv(filename, index_col=None, header=0)
            li.append(df)
        frame = pd.concat(li, axis=0, ignore_index=True)
        frame.to_csv(self.Target['Name']+'_output/'+self.Target['Name']+'_seismicfit.csv',index=None)

    def ExtractGrid(self):
        '''To be added...not necessary (Optional)'''
        print('Please check GridExtract.py for the details of grid extraction.')


    def FitModels(self):
        if not os.path.exists(self.Target['Name']+'_output'):
            os.system('mkdir '+self.Target['Name']+'_output')
            
        #loading grid extract
        total_grid = pd.read_csv('sdb_grid/sdb_grid_extract.csv')
        print('Approximate number of iteration models : ', len(total_grid))
        print('Total number of l1 modes for fitting : ',len(self.Target['PeriodList']))
        a = 'Track,mi,menv,z,y,yc,mass,radius,convcorem,age,zaehbage,teff,logg,luminosity,chisq,total_merit'

        for PL in self.Target['PeriodList']:
            a = f'{a},f{int(np.round(PL))}'
        for PL in self.Target['PeriodList']:
            a = f'{a},f{int(np.round(PL))}_nval'
            
        m_log = [a]
        chuck, chuck_name = 0, 0

        start = timeit.default_timer()
        for track,yc in tqdm(zip(total_grid.Track,total_grid.yc)): 
            #grid_properties
            menv=track[track.find('menv')+4:track.find('_rot')]
            z=track[track.find('_z')+2:track.find('_lvl')]
            y=track[track.find('_y')+2:track.find('_f')]
            mi = track[track.find('mi')+2:track.find('_z')]
            trc = total_grid[(total_grid.Track==track) & (total_grid.yc==yc)]
            #stellar_parameters
            starmass = str(np.round(trc.mass.values[0],6))
            starradius =str(np.round(trc.radius.values[0],6))
            starage = str(np.round(np.min(trc.age.values[0]),6))
            zaehbage = str(np.round(np.min(trc.zaehbage.values[0]),6))
            Teff_model =str(np.round(np.min(trc.teff.values[0]),6))
            logg_model=str(np.round(np.min(trc.logg.values[0]),6))
            luminosity = str(np.round(np.min(trc.luminosity.values[0]),6))
            convcorem = str(np.round(np.min(trc.convcorem.values[0]),6))
            #HRD_chisquarefit
            chisq = (((self.Target['TeffObs']-float(Teff_model))/self.Target['TeffObsErr'])**2)+(((self.Target['loggObs']-float(logg_model))/self.Target['loggObsErr'])**2) 
            chisq = str(np.round(np.min(chisq),3))
            #gyre_period_merit_fit
            path_to_gyre = 'sdb_grid/'+track+'/custom_He'+str(yc)+'_summary.csv'
            gyre_table = pd.read_csv(path_to_gyre)
            gyre_table = gyre_table[(gyre_table.P>=self.Target['GYREPeriodRange'][0]) & (gyre_table.P<self.Target['GYREPeriodRange'][1])]
            ds,dprd,dnval = self.Merit(gyre_table,1)        
            #merit = str(np.round(np.sqrt(np.sum(ds))/len(self.Target['PeriodList']),6))
            merit = str(np.round(np.sum(ds)/len(self.Target['PeriodList']),6))
            #making_data
            b = track +','+mi+','+menv+','+z+','+y+','+str(yc)+','+starmass+','+starradius+','+convcorem+','+starage+','+zaehbage+','+Teff_model+','+logg_model+','+luminosity+','+chisq+','+merit
            for jj2 in dprd:
                b = f'{b},{jj2[0]}'
            for jj2 in dnval:
                b = f'{b},{jj2[0]}'
                
            m_log0 = np.array([b])
            m_log=np.vstack((m_log,m_log0))
            #chunking and saving
            chuck+=1
            if chuck==self.ChunkSize :
                np.savetxt(self.Target['Name']+'_output/'+self.Target['Name']+'_fitgrid_'+str(chuck_name)+'.csv',m_log,fmt="%s")        
                a = 'Track,mi,menv,z,y,yc,mass,radius,convcorem,age,zaehbage,teff,logg,luminosity,chisq,total_merit'
                for PL in self.Target['PeriodList']:
                    a = f'{a},f{int(np.round(PL))}'
                for PL in self.Target['PeriodList']:
                    a = f'{a},f{int(np.round(PL))}_nval'
                    
                m_log = [a]        
                chuck_name +=1
                chuck = 0
        np.savetxt(self.Target['Name']+'_output/'+self.Target['Name']+'_fitgrid_'+str(chuck_name)+'.csv',m_log,fmt="%s")        
        stop = timeit.default_timer()
        print('Done! Run Time: ', stop - start)  
        self.AppendTaskCSV()
        os.system('rm -f '+self.Target['Name']+'_output/'+self.Target['Name']+'_fitgrid_*.csv')
        
    def CheckModels(self,DisplayModel=True,DisplayModelRange=True,DisplayRadialOrder=True):
        pt=pd.read_csv(self.Target['Name']+'_output/'+self.Target['Name']+'_seismicfit.csv')
        print(f'Total number of models : {len(pt)}')
        print(f'Used {len(self.CustomPeriodList)} multiplets')
        print(self.CustomPeriodList)
        totalmerit = np.zeros(1)
        for p in self.CustomPeriodList:
            merit = (pt[f'f{int(np.round(p))}'] - p)**2
            totalmerit = totalmerit + merit
        # Metalicity Fe/H calculations Asplund et al. 2010   
        solar_h1 = 0.7154
        solar_h2 = 1.43e-5
        solar_he3 = 4.49e-5
        solar_he4 = 0.2702551
        solar_x = solar_h1 + solar_h2
        solar_y = solar_he3 + solar_he4
        solar_z = 1.0 - solar_x - solar_y
        pt['FeH'] =  np.log10(pt['z'] / solar_z)

        pt['total_merit'] = totalmerit / len(self.CustomPeriodList)
        pt['since_zaehb'] = (pt.age-pt.zaehbage)/(1.e6)

        tr=pt[(pt['teff']>=self.Target['TeffObs']-(self.Target['Sigma']*self.Target['TeffObsErr']))&(pt['teff']<=self.Target['TeffObs']+(self.Target['Sigma']*self.Target['TeffObsErr']))&(pt['logg']>=self.Target['loggObs']-(self.Target['Sigma']*self.Target['loggObsErr']))&(pt['logg']<=self.Target['loggObs']+(self.Target['Sigma']*self.Target['loggObsErr']))]
        print('Number of models within '+str(self.Target['Sigma'])+' sigma observation : ',len(tr))
        trz = tr.sort_values('total_merit').iloc[:self.NModels] # Only selecting first 1000 models with in the 1 sigma observation limit
        ptz = pt.sort_values('total_merit').iloc[:self.NModels] # Only selecting first 1000 models
        print(f'Merit minimum : {pt.total_merit.min()} Merit maximum : {pt.total_merit.max()} (Over all models.)')

        ptz1 = ptz[['mi', 'menv', 'mass', 'convcorem','radius','z', 'FeH','yc', 'age','since_zaehb','teff','logg','luminosity','total_merit']]
        ptz1.reset_index(drop=True)
        ptz2 = ptz1[ptz1.total_merit<=(self.HowBad/100.)*ptz1.total_merit.min()].reset_index(drop=True) # taking models up to 150% of the minimum merit
        trz1 = trz[['mi', 'menv', 'mass', 'convcorem','radius','z', 'FeH','yc', 'age','since_zaehb','teff','logg','luminosity','total_merit']]
        trz1.reset_index(drop=True)
        trz2 = trz1[trz1.total_merit<=(self.HowBad/100.)*trz1.total_merit.min()].reset_index(drop=True) # taking models up to 150% of the minimum merit

        if DisplayRadialOrder:
            print('_______________________________________________________________________')
            print(f'Displaying best models upto {self.HowBad/100}x over all models : ')
            print('_______________________________________________________________________')
            ptz.reset_index(drop=True)
            ptz3 = ptz[ptz.total_merit<=(self.HowBad/100.)*ptz.total_merit.min()].reset_index(drop=True) # taking models up to 150% of the minimum merit
            print(ptz3)
            
            print('_______________________________________________________________________')
            print(f'Displaying best models upto {self.HowBad/100}x within the observation limit :')
            print('_______________________________________________________________________')
            trz.reset_index(drop=True)
            trz3 = trz[trz.total_merit<=(self.HowBad/100.)*trz.total_merit.min()].reset_index(drop=True) # taking models up to 150% of the minimum merit
            print(trz3)           
        
        if DisplayModel==True:
            print('_______________________________________________________________________')
            print(f'Displaying best models upto {self.HowBad/100}x over all models : ')
            print('_______________________________________________________________________')
            print(ptz2)
            
            print('_______________________________________________________________________')
            print(f'Displaying best models upto {self.HowBad/100}x within the observation limit :')
            print('_______________________________________________________________________')
            print(trz2)
        
        if DisplayModelRange == True:
            #Making modeule for printing and formating ranges of values
            print('_______________________________________________________________________')
            print(f'Displaying parameter range of best models upto {self.HowBad/100}x over all models')
            print('_______________________________________________________________________')
            for i in ptz2.columns:
                if ptz2[i].min() == ptz2[i].max(): print(f'{i} : {ptz2[i].min()}')
                else : print(f'{i} : {ptz2[i].min()} to {ptz2[i].max()}')
            
            print('_______________________________________________________________________')
            print(f'Displaying parameter range of best models up to {self.HowBad/100}x within the observation limit')
            print('_______________________________________________________________________')
            for i in trz2.columns:
                if trz2[i].min() == trz2[i].max(): print(f'{i} : {trz2[i].min()}')
                else : print(f'{i} : {trz2[i].min()} to {trz2[i].max()}')
        #return pt

    def PlotModels(self):
        output_file(str(self.Target['Name'])+'_output/'+str(self.Target['Name'])+'_seismicfit.html',title='Asteroseismic fits of '+str(self.Target['Name']))

        pt=pd.read_csv(str(self.Target['Name'])+'_output/'+str(self.Target['Name'])+'_seismicfit.csv')
        print(f'Total number of models : {len(pt)}')
        print(f'Used {len(self.CustomPeriodList)} multiplets')
        print(self.CustomPeriodList)
        totalmerit = np.zeros(1)
        for p in self.CustomPeriodList:
            merit = (pt[f'f{int(np.round(p))}'] - p)**2
            totalmerit = totalmerit + merit
        # Metalicity Fe/H calculations Asplund et al. 2010   
        solar_h1 = 0.7154
        solar_h2 = 1.43e-5
        solar_he3 = 4.49e-5
        solar_he4 = 0.2702551
        solar_x = solar_h1 + solar_h2
        solar_y = solar_he3 + solar_he4
        solar_z = 1.0 - solar_x - solar_y
        pt['FeH'] =  np.log10(pt['z'] / solar_z)
        pt['total_merit'] = totalmerit / len(self.CustomPeriodList)
        ##############################################
        pt['since_zaehb'] = (pt.age-pt.zaehbage)/(1.e6)
        ##############################################
        pt1=pt[(pt['teff']>=self.Target['TeffObs']-(self.Target['Sigma']*self.Target['TeffObsErr']))&(pt['teff']<=self.Target['TeffObs']+(self.Target['Sigma']*self.Target['TeffObsErr']))&(pt['logg']>=self.Target['loggObs']-(self.Target['Sigma']*self.Target['loggObsErr']))&(pt['logg']<=self.Target['loggObs']+(self.Target['Sigma']*self.Target['loggObsErr']))]
        pt = pt.sort_values('total_merit').iloc[:self.NModels] # Only selecting first 1000 models
        pt1 = pt1.sort_values('total_merit').iloc[:self.NModels] # Only selecting first 1000 models

        pt['logmerit'] = np.log(pt.total_merit)
        pt.teff = pt.teff.astype(float)
        pt.logg = pt.logg.astype(float)
        pt.logmerit = pt.logmerit.astype(float)

        pt1 = pt1[['mi', 'menv', 'mass', 'convcorem','radius','z', 'FeH','yc', 'age','since_zaehb','teff','logg','luminosity','total_merit']]
        pt1.teff = pt1.teff.astype(float)
        pt1.logg = pt1.logg.astype(float)
        pt1.total_merit = pt1.total_merit.astype(float)

        sample = pt.sample(np.shape(pt)[0])
        source = ColumnDataSource(sample)
        sample1 = pt1.sample(np.shape(pt1)[0])
        source1 = ColumnDataSource(sample1)

        TOOLS = "pan,wheel_zoom,box_zoom,reset,tap,box_select"
        cpal = ['#000000','#6B6969','#00ffff','#aaff00'] #bluegrass

        s2 = figure(plot_width=700, plot_height=700,background_fill_color=cpal[0],tools=TOOLS)
        #Use the field name of the column source
        mapper = linear_cmap(field_name='logmerit', palette=Viridis10 ,low=min(pt.logmerit) ,high=max(pt.logmerit))
        tef_log1 = s2.circle(x='teff', y='logg',color=cpal[1],selection_color="red", size=3, source=source, alpha=1,name='map',legend_label='All Models')
        tef_log2 = s2.circle(x='teff', y='logg', line_color=mapper,selection_color="red", size=3, source=source, alpha=1,name='cmap',legend_label='Merit Map')
        color_bar = ColorBar(color_mapper=mapper['transform'], width=15, label_standoff=12)
        s2.add_layout(color_bar, 'right')
        tef_log3 = s2.circle('teff','logg', source=source1, size=5, color=cpal[2],selection_color="blue",alpha=0.8,name='merit',legend_label='Observation')

        #############
        #OBSERVATIONS
        obs_ltext = s2.rect(self.Target['TeffObs'], self.Target['loggObs'], width=2*self.Target['Sigma']*self.Target['TeffObsErr'], height=2*self.Target['Sigma']*self.Target['loggObsErr'], color="#ffff00", height_units='data',width_units='data',alpha=0.50,fill_alpha=0,legend_label='Observation Limit')
        obs_ltext = False
        s2.legend.location = "top_left"
        s2.legend.click_policy="hide"
        s2.legend.orientation= "horizontal"
        s2.legend.background_fill_alpha=0
        s2.legend.border_line_alpha=0
        s2.legend.label_text_color = "white"
        tooltips1 = [('mi','@mi'),('menv','@menv'),('z','@z'),('yc','@yc'),('merit','@total_merit')]
        s2.add_tools(HoverTool(names=['merit','map'],tooltips=tooltips1))
        s2.xaxis.axis_label = 'teff'
        s2.yaxis.axis_label = 'logg'

        titl1 = 'Asteroseismic fits of '+str(self.Target['Name']+'  |     Observation constrained models <======= Table =======> All models')
        s2.add_layout(Title(text=titl1, text_font_size="16pt"), 'above')
        s2.y_range.flipped = True
        s2.x_range.flipped = True
        s2.xgrid.grid_line_alpha = 0.2
        s2.ygrid.grid_line_alpha = 0.2

        pt1 = pt1[['mi', 'menv', 'mass', 'convcorem','radius','z', 'FeH','yc', 'age','since_zaehb','teff','logg','luminosity','total_merit']]
        sourceTableSummary1 = ColumnDataSource(pt1)
        formatter =  HTMLTemplateFormatter()
        Columns1 = [TableColumn(field=colIndex, title=colIndex, formatter=formatter) for colIndex in pt1.columns] 
        data_table1 = DataTable(columns=Columns1, source=source1, index_position = 0, width=700, height=700,selectable=True,editable=False,fit_columns=True) 

        pt = pt[['mi', 'menv', 'mass', 'convcorem','radius','z', 'FeH','yc', 'age','since_zaehb','teff','logg','luminosity','total_merit']]
        sourceTableSummary0 = ColumnDataSource(pt)
        formatter =  HTMLTemplateFormatter()
        Columns0 = [TableColumn(field=colIndex, title=colIndex, formatter=formatter) for colIndex in pt.columns] 
        data_table0 = DataTable(columns=Columns0, source=source, index_position = 0, width=700, height=700,selectable=True,editable=False,fit_columns=True) 
        l = layout(s2,[data_table1,data_table0],sizing_mode='stretch_both')
        show(l)
 
        

## How to run each module
* The target parameters are filled in to python dictionary.
* Use a unique target name for each Target model (It will create an output folder with the name)
* Add the observed/expected temparature and surface gravity ranges.
* List the all the possible multiplet input periods for fitting in the PeriodList

## Creating class object and fitting Models
* ```TeffObs``` and ```TeffObsErr``` in kK
* ```Sigma``` : Observation error limit
* ```PeriodList``` : add all possible l1 multiplets to this list before the fitting.
* ```GYREPeriodRange``` : GYRE period min max range in seconds # The range of observed period to be taken from the GYRE models.
* ```FitModels``` will create a ```xxxxx_seismicfit.csv``` file in the out put folder, it has all the information about the fitting.

In [ ]:
'''
#Example B3
Target = {   'Name'      : 'B3_280223',
            'TeffObs'    : 23.54,
            'TeffObsErr' : 0.21,
            'loggObs'    : 5.311,
            'loggObsErr' : 0.035,
            'Sigma'      : 1.0, 
            'PeriodList' : [7683.263,7201.584,5282.077,5039.728,4794.415,4519.496,4292.176,4047.01,3283.639,3032.43],
            'GYREPeriodRange' : [2000,10000] #Gyre period min max range in seconds # The range of observed period to be taken from the GYRE models.
        }

#Example B4
Target = {   'Name'      : 'B4_280223',
            'TeffObs'    : 25.29,
            'TeffObsErr' : 0.3,
            'loggObs'    : 5.510,
            'loggObsErr' : 0.043,
            'Sigma'      : 1, 
            'PeriodList' : [4610.602,4366.771,3435.5289,3149.775,2914.9386,2679.08565,2466.66761,2263.4784],
            'GYREPeriodRange' : [2000,10000] #Gyre period min max range in seconds # The range of observed period to be taken from the GYRE models.
        }
        
#Example KIC2991403
Target = {   'Name'      : 'KIC2991403_280223',
            'TeffObs'    : 27.3,
            'TeffObsErr' : 0.2,
            'loggObs'    : 5.430,
            'loggObsErr' : 0.03,
            'Sigma'      : 1, 
            'PeriodList' : [6362.558,5123.517,4337.397,3781.592,3512.240,3239.084,2986.6839,2709.985],
            'GYREPeriodRange' : [2000,10000] #Gyre period min max range in seconds # The range of observed period to be taken from the GYRE models.
        }
'''

Target = {   'Name'      : 'KIC11179657_280223',
            'TeffObs'    : 26.0,
            'TeffObsErr' : 0.8,
            'loggObs'    : 5.14,
            'loggObsErr' : 0.13,
            'Sigma'      : 1, 
            'PeriodList' : [6835.44,6612.32,5362.571, 5109.267, 4840.46,3777.025,3513.349, 3239.849, 2965.8311, 2709.807],
            'GYREPeriodRange' : [2000,10000] #Gyre period min max range in seconds # The range of observed period to be taken from the GYRE models.
        }
GridFitObj = sdBMESAGYRE(Target) # Create the sdB MESA GYRE fit object use this command 
GridFitObj.FitModels() # To fit the MESA GYRE Models use this command
#sdBMESAGYRE(Target).FitModels() # We could also use this command in one line

## Checking the models
* Use the ```CustomPeriod``` list to add and remove periods.
* Do not change PeriodList in the target object 
* Change the ```Sigma``` to change the observation error limit
* It will print out the top results up to 150% worst solutions. (150 % is by default)
* You can change worstness percentage by adding a parameter called ```HowBad``` in the object example : ```sdBMESAGYRE(HowBad=200)```  
* You could select the best solution range by adding this parameters to the object ```NModels```. for example : ```sdBMESAGYRE(NModels=5000)``` (1000 is default)
* 
* We can display best models or their range of parameters by changing variables in the ```CheckModels``` module.
    * To display best models in each cases you can use ```DisplayModel=True```. It's True by default.
    * To display only the parametrer ranges you can use ```DisplayModelRange=True```. It's True by default. 

In [ ]:
Target = {   'Name'      : 'KIC11179657_280223',
            'TeffObs'    : 26.0,
            'TeffObsErr' : 0.8,
            'loggObs'    : 5.14,
            'loggObsErr' : 0.13,
            'Sigma'      : 1, 
            'PeriodList' : [6835.44,6612.32,5362.571, 5109.267, 4840.46,3777.025,3513.349, 3239.849, 2965.8311, 2709.807],
            'GYREPeriodRange' : [2000,10000] #Gyre period min max range in seconds # The range of observed period to be taken from the GYRE models.
        }
#CustomPeriod = [00000010,00000009,00000008,00000007,00000006,00000005,00000004,0000003,00000002,0000001]
CustomPeriod = [6835.44,6612.32,5362.571, 5109.267, 4840.46,3777.025,3513.349, 3239.849, 2965.8311, 2709.807] #1-10

#Custom Period, Only change CustomPeriod,  Do not change PeriodList in the target object 
sdBObj = sdBMESAGYRE(Target,CustomPeriodList=CustomPeriod,NModels=1000,HowBad=150)
#sdBObj.FitModels() # fit only once
sdBObj.CheckModels(DisplayModel=False, DisplayModelRange=True, DisplayRadialOrder=True) # To test the model fits with custom Period use this command
#sdBObj.PlotModels()

In [ ]:
#Example
Target = {   'Name'      : 'B3_280223',
            'TeffObs'    : 23.54,
            'TeffObsErr' : 0.21,
            'loggObs'    : 5.311,
            'loggObsErr' : 0.035,
            'Sigma'      : 1.0, 
            'PeriodList' : [7683.263,7201.584,5282.077,5039.728,4794.415,4519.496,4292.176,4047.01,3283.639,3032.43],
            'GYREPeriodRange' : [2000,10000] #Gyre period min max range in seconds # The range of observed period to be taken from the GYRE models.
        }

#CustomPeriod = [00000010,00000009,00000008,00000007,00000006,00000005,00000004,0000003,00000002,0000001]
CustomPeriod = [7683.263,7201.584,5282.077,5039.728,4794.415,4519.496,4292.176,4047.01,3283.639,3032.43] #1-10

#Custom Period, Only change CustomPeriod,  Do not change PeriodList in the target object 
sdBObj = sdBMESAGYRE(Target,CustomPeriodList=CustomPeriod,NModels=1000,HowBad=150)
#sdBObj.FitModels() # fit only once
sdBObj.CheckModels(DisplayModel=False, DisplayModelRange=True, DisplayRadialOrder=True) # To test the model fits with custom Period use this command
#sdBObj.PlotModels()

In [ ]:
#Example B4
Target = {   'Name'      : 'B4_280223',
            'TeffObs'    : 25.29,
            'TeffObsErr' : 0.3,
            'loggObs'    : 5.510,
            'loggObsErr' : 0.043,
            'Sigma'      : 1, 
            'PeriodList' : [4610.602,4366.771,3435.5289,3149.775,2914.9386,2679.08565,2466.66761,2263.4784],
            'GYREPeriodRange' : [2000,10000] #Gyre period min max range in seconds # The range of observed period to be taken from the GYRE models.
        }

#CustomPeriod =[00000001,00000002,000000003,00000004,000000005,0000000006,0000000007,000000008]
CustomPeriod = [4610.602,4366.771,3435.5289,3149.775,2914.9386,2679.08565,2466.66761,2263.4784]

sdBObj = sdBMESAGYRE(Target,CustomPeriodList=CustomPeriod,NModels=1000,HowBad=150)
#sdBObj.FitModels() # fit only once
sdBObj.CheckModels() # To test the model fits with custom Period use this command
#sdBObj.PlotModels()

In [ ]:
#Example KIC2991403
Target = {   'Name'      : 'KIC2991403_280223',
            'TeffObs'    : 27.3,
            'TeffObsErr' : 0.2,
            'loggObs'    : 5.430,
            'loggObsErr' : 0.03,
            'Sigma'      : 1, 
            'PeriodList' : [6362.558,5123.517,4337.397,3781.592,3512.240,3239.084,2986.6839,2709.985],
            'GYREPeriodRange' : [2000,10000] #Gyre period min max range in seconds # The range of observed period to be taken from the GYRE models.
        }


#CustomPeriod = [00000008,00000007,00000006,00000005,00000004,00000003,000000002,00000001]
CustomPeriod = [6362.558,5123.517,4337.397,3781.592,3512.240,3239.084,2986.6839,2709.985]

sdBObj = sdBMESAGYRE(Target,CustomPeriodList=CustomPeriod,NModels=1000,HowBad=150)
#sdBObj.FitModels() # fit only once
sdBObj.CheckModels() # To test the model fits with custom Period use this command
#sdBObj.PlotModels()

In [ ]:
#Example KIC11179657
Target = {   'Name'      : 'KIC11179657_280223',
            'TeffObs'    : 26.0,
            'TeffObsErr' : 0.8,
            'loggObs'    : 5.14,
            'loggObsErr' : 0.13,
            'Sigma'      : 1, 
            'PeriodList' : [6835.44,6612.32,5362.571, 5109.267, 4840.46,3777.025,3513.349, 3239.849, 2965.8311, 2709.807],
            'GYREPeriodRange' : [2000,10000] #Gyre period min max range in seconds # The range of observed period to be taken from the GYRE models.
        }

#CustomPeriod = [0000010, 0000009, 00000008, 00000007, 0000006, 00000005, 00000004, 00000003, 000000002, 00000001]
CustomPeriod = [6835.44, 6612.32, 5362.571, 5109.267, 4840.46, 3777.025, 3513.349, 3239.849, 2965.8311, 2709.807]

sdBObj = sdBMESAGYRE(Target,CustomPeriodList=CustomPeriod,NModels=1000,HowBad=150)
#sdBObj.FitModels() # fit only once
sdBObj.CheckModels() # To test the model fits with custom Period use this command
#sdBObj.PlotModels()

In [ ]:

Star = 'KIC11179657'

if Star=='B3':
    #B3 model dictionary
    ModelDict = {'model1' : [5039.728,4794.415,3283.639,3032.43],
            'model2' : [5282.077,5039.728,4794.415,3283.639,3032.43],
            'model3' : [5039.728,4794.415,4292.176,4047.01,3283.639,3032.43],
            'model4' : [5282.077,5039.728,4794.415,4292.176,4047.01,3283.639,3032.43],
            'model5' : [5282.077,5039.728,4794.415,4519.496,4292.176,4047.01,3283.639,3032.43],
            'model6' : [7683.263,7201.584,5282.077,5039.728,4794.415,4292.176,4047.01,3283.639,3032.43],	
            'model7' : [7201.584,5282.077,5039.728,4794.415,4519.496,4292.176,4047.01,3283.639,3032.43],
            'model8' : [7683.263,7201.584,5282.077,5039.728,4794.415,4519.496,4292.176,4047.01,3283.639,3032.43],
    }
    # B3 
    Target = {   'Name'      : 'B3_280223',
                'TeffObs'    : 23.54,
                'TeffObsErr' : 0.21,
                'loggObs'    : 5.311,
                'loggObsErr' : 0.035,
                'Sigma'      : 1.0, 
                'PeriodList' : [7683.263,7201.584,5282.077,5039.728,4794.415,4519.496,4292.176,4047.01,3283.639,3032.43],
                'GYREPeriodRange' : [2000,10000] #Gyre period min max range in seconds # The range of observed period to be taken from the GYRE models.
    }

elif Star=='B4':
    #B4 model dictionary
    ModelDict = {'model1' : [4610.602,3149.775,2914.9386],
                'model2' : [4610.602,3149.775,2914.9386,2679.08565],
                'model3' : [4610.602,3149.775,2914.9386,2679.08565,2466.66761],
                'model4' : [4610.602,3435.5454,2914.9386,2679.08565,2466.66761],
                'model5' : [4610.602,3435.5289,3149.775,2914.9386,2679.08565,2466.66761],
                'model6' : [4610.602,4366.771,3435.5289,3149.775,2914.9386,2679.08565,2466.66761],
                'model7' : [4610.602,4366.771,3435.5289,3149.775,2914.9386,2679.08565,2466.66761,2263.4784]
    }
    # B4
    Target = {   'Name'      : 'B4_280223',
                'TeffObs'    : 25.29,
                'TeffObsErr' : 0.3,
                'loggObs'    : 5.510,
                'loggObsErr' : 0.043,
                'Sigma'      : 1, 
                'PeriodList' : [4610.602,4366.771,3435.5289,3149.775,2914.9386,2679.08565,2466.66761,2263.4784],
                'GYREPeriodRange' : [2000,10000] #Gyre period min max range in seconds # The range of observed period to be taken from the GYRE models.
    }
elif Star == 'KIC2991403':
    #KIC2991403 model dictionary
    ModelDict = { 'model1' : [6362.558,5123.517,3239.084,2986.6839],
                    'model2' : [6362.558,5123.517,3512.240,3239.084,2986.6839],
                    'model3' : [6362.558,5123.517,4337.397,3512.240,3239.084,2986.6839,2709.985],
                    'model4' : [6362.558,5123.517,4337.397,3781.592,3239.084,2986.6839,2709.985],
                    'model5' : [6362.558,5123.517,4337.397,3781.592,3512.240,3239.084,2986.6839],
                    'model6' : [6362.558,5123.517,4337.397,3781.592,3512.240,3239.084,2986.6839,2709.985]
                    }
    # KIC2991403
    Target = {   'Name'      : 'KIC2991403_280223',
        'TeffObs'    : 27.3,
        'TeffObsErr' : 0.2,
        'loggObs'    : 5.430,
        'loggObsErr' : 0.03,
        'Sigma'      : 1, 
        'PeriodList' : [6362.558,5123.517,4337.397,3781.592,3512.240,3239.084,2986.6839,2709.985],
        'GYREPeriodRange' : [2000,10000] #Gyre period min max range in seconds # The range of observed period to be taken from the GYRE models.
    }
elif Star == 'KIC11179657':
    # KIC11179657 model dictionary
    ModelDict = { 'model1' : [6835.44, 5109.267, 3513.349, 3239.849],	
                    'model2' : [6835.44, 5109.267, 3777.025, 3513.349, 3239.849],
                    'model3' : [3777.025, 3513.349, 3239.849, 2965.8311, 2709.807],
                    'model4' : [6835.44, 5362.571, 5109.267, 3777.025, 3513.349, 3239.849, 2965.8311],
                    'model5' : [6835.44, 5362.571, 5109.267, 4840.46, 3777.025, 3513.349, 3239.849, 2965.8311],
                    'model6' : [5362.571, 5109.267, 4840.46, 3777.025, 3513.349, 3239.849, 2965.8311, 2709.807],
                    'model7' : [6835.44, 6612.32, 5362.571, 5109.267, 4840.46, 3777.025, 3513.349, 3239.849, 2965.8311],
                    'model8' : [6835.44, 6612.32, 5362.571, 5109.267, 4840.46, 3777.025, 3513.349, 3239.849, 2965.8311, 2709.807]
    }    
    # KIC11179657
    Target = {  'Name'       : 'KIC11179657_280223',
                'TeffObs'    : 26.0,
                'TeffObsErr' : 0.8,
                'loggObs'    : 5.14,
                'loggObsErr' : 0.13,
                'Sigma'      : 1, 
                'PeriodList' : [6835.44,6612.32,5362.571, 5109.267, 4840.46,3777.025,3513.349, 3239.849, 2965.8311, 2709.807],
                'GYREPeriodRange' : [2000,10000] #Gyre period min max range in seconds # The range of observed period to be taken from the GYRE models.
    }



# Visualize the filtered solutions and cases
* The ```PlotModels``` will make an html file located in the output folder containing the filtered results interactively plotted
* The plot has a main plot panel with the Teff and logg axis
* The best solutions from all models and observation constrained models are presented.

* The bottom panel has two tables, 
    * Table on the right has best solutions from all observations.
    * Table on the left has best solutions from Observation limited models.
    * You can sort them based on any column by clicking on the column header.

* The plots will highlight upon clicking on the cells on the table.
    * Red high lights are models from all the observations
    * Blue highlights are modles constrained by observation.
    * There is also a merit map for all the models.
    * The observations with Sigma limit is represented as a yellow rectangle
    * click hide each legends based on use.

* Use the ```CustomPeriod``` list to add and remove periods.
* Change the ```Sigma``` to change the observation error limit
* It will print out the top results up to 150% worst solutions. 
* You can change worstness the percentage by adding a parameter called ```HowBad``` in the object example : ```sdBMESAGYRE(HowBad=200)```  
* You could select the best solution range by adding this parameters to the object ```NModels```. for example : ```sdBMESAGYRE(NModels=5000)```


In [ ]:
# Example
Target = {   'Name'      : 'KIC2991403_280223',
            'TeffObs'    : 27.3,
            'TeffObsErr' : 0.2,
            'loggObs'    : 5.43,
            'loggObsErr' : 0.03,
            'Sigma'      : 1, 
            'PeriodList' : [6362.558,5123.517,4337.397,3781.592,3512.240,3239.084,2986.6839,2709.985],
            'GYREPeriodRange' : [2000,10000] #Gyre period min max range in seconds # The range of observed period to be taken from the GYRE models.
        }
CustomPeriod = [6362.558,5123.517,4337.397,3781.592,3512.240,3239.084,2986.6839,2709.985] # Custom Period, Only change CustomPeriod,  Do not change PeriodList in the target object 
sdBMESAGYRE(Target,CustomPeriodList=CustomPeriod,NModels=1000).PlotModels()

In [ ]:
# Example 
# Plotting best 5000 models with a 3 Sigma error limit. 
# Only taking four multiplets to Test 
Target = {   'Name'      : 'KIC2991403',
            'TeffObs'    : 27.3,
            'TeffObsErr' : 0.2,
            'loggObs'    : 5.43,
            'loggObsErr' : 0.03,
            'Sigma'      : 3, 
            'PeriodList' : [6362.558,5123.517,4337.397,3781.592,3512.240,3239.084,2986.6839,2709.985],
            'GYREPeriodRange' : [2000,10000] #Gyre period min max range in seconds # The range of observed period to be taken from the GYRE models.
        }
CustomPeriod = [6362.558,5123.517,4337.397,3781.592] # Custom Period, Only change CustomPeriod,  Do not change PeriodList in the target object 
sdBMESAGYRE(Target, CustomPeriodList=CustomPeriod,NModels=5000).PlotModels()